<h2>Install 시작!!!

In [1]:
!pip install selenium beautifulsoup4 requests webdriver-manager


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\jrnee\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [43]:
import re
import unicodedata
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import requests
import os


<h3> URL 생성함수

In [44]:
# URL 생성 함수
def get_url(gender='', category=''):
    base_url = 'https://www.musinsa.com/category'
    params = {
        'gf': gender,
        'category': category
    }
    query_string = '&'.join([f"{key}={value}" for key, value in params.items() if value])
    return f"{base_url}/{category}?{query_string}"

# 파일 이름 정리 함수
def clean_filename(filename):
    # Unicode 정규화 (한글 포함 문자 처리 안정성 향상)
    filename = unicodedata.normalize('NFC', filename)
    # 파일 이름에 허용되지 않는 문자 제거
    filename = re.sub(r'[\/:*?"<>|]', '_', filename)
    # 공백과 특수 문자 제거
    filename = re.sub(r'[\n\r\t]', '', filename)
    return filename.strip()

<h3> 이미지 크롤링

In [45]:
# 이미지 크롤링 함수
def crawl_images(driver, gender, category, csv_writer):
    # 성별 및 카테고리 이름 매핑
    gender_map = {'M': '남성', 'F': '여성'}
    category_map = {'002022': '후드집업', '003002': '데님팬츠', '002007': '코트'}

    gender_name = gender_map.get(gender, gender)
    category_name = category_map.get(category, category)

    # 스크롤 설정
    scroll_pause_time = 2
    screen_height = driver.execute_script("return window.screen.height;")
    i = 1

    downloaded_count = 0
    max_downloads = 3  # 최대 다운로드 수 설정

    # 저장 디렉토리 생성
    base_dir = "C:\\Users\\jrnee\\Desktop\\무신사크롤링"
    output_dir = os.path.join(base_dir, gender_name, category_name)
    os.makedirs(output_dir, exist_ok=True)

    while downloaded_count < max_downloads:
        driver.execute_script(f"window.scrollTo(0, {screen_height}*{i});")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight;")
        if new_height == screen_height * i:
            break
        i += 1

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 상품 목록 추출
        product_elements = soup.select('a[aria-label="상품 상세로 이동"]')
        for product_element in product_elements:
            try:
                # 상품 상세 페이지 URL
                href = product_element.get('href')
                if href.startswith('http'):  # 절대 경로인 경우
                    detail_url = href
                else:  # 상대 경로인 경우
                    detail_url = f"https://www.musinsa.com{href}"

                # 이미지 태그 추출
                img_tag = product_element.select_one('img')
                if not img_tag:
                    continue

                img_url = img_tag.get('src')
                raw_img_name = img_tag.get('alt', 'unknown')
                
                # 파일 이름 정리
                img_name = clean_filename(raw_img_name)
                
                # 이름이 유효하지 않은 경우 기본값 설정
                if not img_name.strip():
                    img_name = f"image_{downloaded_count}"

                if not img_url or not img_name:
                    continue

                if downloaded_count >= max_downloads:
                    print(f"Downloaded {downloaded_count} images. Stopping further downloads.")
                    return

                # 이미지 다운로드
                img_response = requests.get(img_url)
                img_path = os.path.join(output_dir, f"{img_name}.jpg")
                with open(img_path, 'wb') as f:
                    f.write(img_response.content)

                # CSV 파일에 기록
                csv_writer.writerow([img_name, detail_url])
                downloaded_count += 1

                print(f"Downloaded: {img_name}, Detail URL: {detail_url}")
            except Exception as e:
                print(f"Failed to process product: {e}")

    print(f"Downloaded {downloaded_count} images to {output_dir}")

<h3> 크롤링 시작

In [46]:
# 메인 함수
def main():
    genders = ['M', 'F']  # 성별 리스트 (M: 남성, F: 여성)
    categories = ['002022', '003002', '002007']  # 카테고리 리스트 (002022: 후드집업, 003002: 데님팬츠, 002007: 코트)

    # CSV 파일 생성
    csv_file_path = "C:\\Users\\jrnee\\Desktop\\무신사크롤링\\musinsa_data.csv"
    with open(csv_file_path, mode='w', newline='', encoding='utf-8-sig') as csv_file:  # utf-8-sig 적용
        csv_writer = csv.writer(csv_file)
        # CSV 헤더 작성
        csv_writer.writerow(["Image Name", "Product Detail URL"])

        # 웹 드라이버 설정
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

        try:
            for gender in genders:
                for category in categories:
                    url = get_url(gender=gender, category=category)
                    print(f"Generated URL: {url}")
                    driver.get(url)
                    crawl_images(driver, gender, category, csv_writer)
        finally:
            driver.quit()

if __name__ == "__main__":
    main()

Generated URL: https://www.musinsa.com/category/002022?gf=M&category=002022
Downloaded: 데님 워싱 후드 집업 인디고, Detail URL: https://www.musinsa.com/products/3439576
Downloaded: 루트 썬그립 바람막이 Navy (S23MURWB22), Detail URL: https://www.musinsa.com/products/3030990
Downloaded: 데님 워싱 후드 집업 차콜, Detail URL: https://www.musinsa.com/products/3439575
Downloaded 3 images. Stopping further downloads.
Generated URL: https://www.musinsa.com/category/003002?gf=M&category=003002
Downloaded: 씬핀 Dry selvage comfort _ 111868, Detail URL: https://www.musinsa.com/products/4324275
Downloaded: Sleepy Sixten Joey Replica _ 112947, Detail URL: https://www.musinsa.com/products/4324209
Downloaded: 그림팀 David Replica _ 112375, Detail URL: https://www.musinsa.com/products/4324264
Downloaded 3 images. Stopping further downloads.
Generated URL: https://www.musinsa.com/category/002007?gf=M&category=002007
Downloaded: 울 블렌드 오버사이즈 머프 발마칸 코트 [차콜 그레이], Detail URL: https://www.musinsa.com/products/4209270
Downloaded: 캐시미어 히든 싱글 코트

<h3>Done!!